<a href="https://colab.research.google.com/github/Stealth-Food-Exploration/web-application/blob/main/Dataset_Research.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = '6000-indian-food-recipes-dataset:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F934844%2F1584379%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240401%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240401T190131Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D77c611546bcf5655f298d61b2cd1ed4fdaf8cabfd02ee182c0be7554c4839c26d94d3b41d9315c89999d7da6a927634eba1dc48f4f490848dde165ad62d17a5519feb26fb7973584756ee6ce6103cd6e176668639005ad6c7b7851a9ef63e2f9c5c04a4c9e12c82eb658a3f0f989d8493ac054082babdf99c522251de52eddccd79f774720684b3e83f5900c39ad0f05b16b84828c8c7c2b2fca887ece4213220feceb968677ac4f98b419de79aca30f0ec61e9bb4bade5a7c1da60a3acf03ea2ea4be427e6366217c405f776fbb71afc9936af8a2ca9007308f86498c9fa9c6be76ac8552d7d04842dd6590e847b6674eac6aa519220101a8d1e41ac3483adc,indian-food-images-dataset:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F1243366%2F4077749%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240401%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240401T190131Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3Db37bff5a0a30a7b348c6dab5134c98250efad7072c21f6db5224bc4555c9fd52446f78ecf9d1b72d43b6440addf4da2897ed14a33115dc7e7b128b4979a29ee92b505ab0d9edfbb453f62de2c84030d679b7e5bb7a518076b5cb81779087a9ded4c6053fabd799fc6b6b137ce5ff1168fdf096f0cfe1378e136aeeff532fc112754c90897e9ecd255dab2129542ef9ad5d7dc0685f3a60637d82aa935df3d9ce7086ad965ae2cf8ae86bd8bce705a5ab0a22bb5451fae825b55350158c30629939e6f0b743b035a4de490dc19c97a975c6ae8c3db834a9f14d4a841e6e80d271588ed42c552c40a1a031711fc2af8cb8e8f282e00448e4353691dfcb48555d87,indian-food-and-its-recipes-dataset-with-images:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F2440705%2F4130977%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240401%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240401T190131Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D8ff4651b09eb5ca1c1ec7c7c685a4457808fc48267a9f8b1f77938cfea03ee8d599ba251e875ffcfe485744d8ed6cd1d0569eb73c64a0dbc4188449cb05a5ae1a0803a85527abeba0d054e1af583e311594349aad893f5c250c091070504d57103c8113620eb96b0a64483a356b7a76c30a113393ffe6edc77e7ed56ecaa5ba8c4ef21924edb80bcce326fba6c630df9c5c901208e22f75b04bcc82655ef40038611503ba320a0d2bbefe3fe2caf2b985f1c2d01e34759f1bb3efec81185fd1db2a936a8946be3aa8c00cbf5031fab54b9c6679a740ab169cfa1b132c306bdc56063bf68ff6ab3f849621aded04581aa724c4baba983c7a2adab420ded27c89b'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


In [ ]:
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

## 1. Indian Food and Its Recipes Dataset (With Images)


Context :-
This data was created to build a Deep Learning based image to recipes model which can provide ingredients and recipe of the dish once the image is uploaded

Source :-
The data is scrapped from website (Link :- https://www.archanaskitchen.com/) and images were downloaded from the image download code using python. You can find both the code below
Github Link :- https://github.com/websitecreatr99/Web-Scrapping-For-Cuisines.git

In [ ]:
df = pd.read_csv("/kaggle/input/indian-food-and-its-recipes-dataset-with-images/cuisines.csv")

In [ ]:
df.count()

name            4236
image_url       4236
description     4236
cuisine         4230
course          4198
diet            4199
prep_time       4226
ingredients     4236
instructions    4236
dtype: int64

## 2. Indian Food Images Dataset

Source: This Dataset is created from Google Images: https://images.google.com/. If you want to learn more, you can visit the Website.

Indian cuisine consists of a variety of regional and traditional cuisines native to the Indian subcontinent. Given the diversity in soil, climate, culture, ethnic groups, and occupations, these cuisines vary substantially and use locally available spices, herbs, vegetables, and fruits. Indian food is also heavily influenced by religion, in particular Hinduism, cultural choices, and traditions. Centuries of Islamic rule, particularly by the Mughals, also introduced dishes like samosa and pilaf.

Historical events such as invasions, trade relations, and colonialism have played a role in introducing certain foods to this country. The Columbian discovery of the New World brought a number of new vegetables and fruit to India. A number of these such as the potato, tomatoes, chilies, peanuts, and Guava have become staples in many regions of India. Indian cuisine has shaped the history of international relations; the spice trade between India and Europe was the primary catalyst for Europe's Age of Discovery. Spices were bought from India and traded around Europe and Asia. Indian cuisine has influenced other cuisines across the world, especially those from Europe (especially Britain), the Middle East, Southern African, East Africa, Southeast Asia, North America, Mauritius, Fiji, Oceania, and the Caribbean.

Content
In this Dataset, we have 4000 Indian Food Images in 80 different categories or classes.

In [ ]:
import os
File_object = open("/kaggle/input/indian-food-images-dataset/List of Indian Foods.txt", "r")
images = File_object.read()
print(images)


adhirasam

aloo_gobi

aloo_matar

aloo_methi

aloo_shimla_mirch

aloo_tikki

anarsa

ariselu

bandar_laddu

basundi

bhatura

bhindi_masala

biryani

boondi

butter_chicken

chak_hao_kheer

cham_cham

chana_masala

chapati

chhena_kheeri

chicken_razala

chicken_tikka

chicken_tikka_masala

chikki

daal_baati_churma

daal_puri

dal_makhani

dal_tadka

dharwad_pedha

doodhpak

double_ka_meetha

dum_aloo

gajar_ka_halwa

gavvalu

ghevar

gulab_jamun

imarti

jalebi

kachori

kadai_paneer

kadhi_pakoda

kajjikaya

kakinada_khaja

kalakand

karela_bharta

kofta

kuzhi_paniyaram

lassi

ledikeni

litti_chokha

lyangcha

maach_jhol

makki_di_roti_sarson_da_saag

malapua

misi_roti

misti_doi

modak

mysore_pak

naan

navrattan_korma

palak_paneer

paneer_butter_masala

phirni

pithe

poha

poornalu

pootharekulu

qubani_ka_meetha

rabri

ras_malai

rasgulla

sandesh

shankarpali

sheer_korma

sheera

shrikhand

sohan_halwa

sohan_papdi

sutar_feni

unni_appam


### 3. 6000+ Indian Food Recipes Dataset



The dataset has following fields (self-explanatory) - ['RecipeName', 'TranslatedRecipeName', 'Ingredients', 'TranslatedIngredients', 'Prep', 'Cook', 'Total', 'Servings', 'Cuisine', 'Course', 'Diet', 'Instructions', 'TranslatedInstructions']. The datset contains a csv and a xls file. Sometimes, the content in Hindi is not visible in the csv format.

You might be wondering what the columns with the prefix 'Translated' are. So, a lot of entries in the dataset were in Hindi language. To take care of such entries and translating them to English for consistency, I went ahead and used 'googletrans'. It is a python library that implements Google Translate API underneath.

The code for the crawler, cleaning and transformation is on my Github repository (@kanishk307).

The dataset has been created using Archana's Kitchen Website. It is a great website and hosts a ton of useful content. You should definitely consider viewing it if you are interested.

In [ ]:
data = pd.read_csv("/kaggle/input/6000-indian-food-recipes-dataset/IndianFoodDatasetCSV.csv")
data.head()

,Srno,RecipeName,TranslatedRecipeName,Ingredients,TranslatedIngredients,PrepTimeInMins,CookTimeInMins,TotalTimeInMins,Servings,Cuisine,Course,Diet,Instructions,TranslatedInstructions,URL
0,1,Masala Karela Recipe,Masala Karela Recipe,"6 Karela (Bitter Gourd/ Pavakkai) - deseeded,S...","6 Karela (Bitter Gourd/ Pavakkai) - deseeded,S...",15,30,45,6,Indian,Side Dish,Diabetic Friendly,"To begin making the Masala Karela Recipe,de-se...","To begin making the Masala Karela Recipe,de-se...",https://www.archanaskitchen.com/masala-karela-...
1,2,टमाटर पुलियोगरे रेसिपी - Spicy Tomato Rice (Re...,Spicy Tomato Rice (Recipe),"2-1/2 कप चावल - पका ले,3 टमाटर,3 छोटा चमच्च बी...","2-1 / 2 cups rice - cooked, 3 tomatoes, 3 teas...",5,10,15,3,South Indian Recipes,Main Course,Vegetarian,टमाटर पुलियोगरे बनाने के लिए सबसे पहले टमाटर क...,"To make tomato puliogere, first cut the tomato...",http://www.archanaskitchen.com/spicy-tomato-ri...
2,3,Ragi Semiya Upma Recipe - Ragi Millet Vermicel...,Ragi Semiya Upma Recipe - Ragi Millet Vermicel...,"1-1/2 cups Rice Vermicelli Noodles (Thin),1 On...","1-1/2 cups Rice Vermicelli Noodles (Thin),1 On...",20,30,50,4,South Indian Recipes,South Indian Breakfast,High Protein Vegetarian,"To begin making the Ragi Vermicelli Recipe, fi...","To begin making the Ragi Vermicelli Recipe, fi...",http://www.archanaskitchen.com/ragi-vermicelli...
3,4,Gongura Chicken Curry Recipe - Andhra Style Go...,Gongura Chicken Curry Recipe - Andhra Style Go...,"500 grams Chicken,2 Onion - chopped,1 Tomato -...","500 grams Chicken,2 Onion - chopped,1 Tomato -...",15,30,45,4,Andhra,Lunch,Non Vegeterian,To begin making Gongura Chicken Curry Recipe f...,To begin making Gongura Chicken Curry Recipe f...,http://www.archanaskitchen.com/gongura-chicken...
4,5,आंध्रा स्टाइल आलम पचड़ी रेसिपी - Adrak Chutney ...,Andhra Style Alam Pachadi Recipe - Adrak Chutn...,"1 बड़ा चमच्च चना दाल,1 बड़ा चमच्च सफ़ेद उरद दाल,2...","1 tablespoon chana dal, 1 tablespoon white ura...",10,20,30,4,Andhra,South Indian Breakfast,Vegetarian,आंध्रा स्टाइल आलम पचड़ी बनाने के लिए सबसे पहले ...,"To make Andhra Style Alam Pachadi, first heat ...",https://www.archanaskitchen.com/andhra-style-a...
